In [1]:
import aiohttp
from pathlib import Path
import pandas as pd
import utils
import asyncio
import math
import utils
import numpy as np
import os

df = pd.read_csv(utils.BOOKS_DATA_BOOKS_PROCESSED)

In [2]:
no_photo_link = 'https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png'
base_url = 'https://images.gr-assets.com/books/'
df['image_url'] = df['image_url'].replace({no_photo_link: np.nan})

os.makedirs(utils.BOOKS_DATA_IMAGES, exist_ok=True)

In [3]:
async def load_and_save_img(image_url, session):
    if image_url is np.nan:
        return
    filename = image_url.replace(base_url, '').replace('/', '-')
    path = utils.BOOKS_DATA_IMAGES / filename
    # is_file checks if file exists
    if path.is_file() == True:
        return
    async with session.get(image_url) as response:
        content = await response.read()
    if response.status == 200:
        with open(path, 'wb') as file:
            file.write(content)
    else:
        print(f'Status code : {response.status} for {image_url}')

In [4]:
async def save_images(start, count, session, urls):
    _ = [await load_and_save_img(url, session) for url in urls[start:start + count]]

In [5]:
async def run_tasks(urls):
    batch_size = 100
    end = math.ceil(df.shape[0] / batch_size)
    async with aiohttp.ClientSession() as session:
        tasks = [save_images(i * batch_size, batch_size, session, urls) for i in range(end)]
        results = await asyncio.gather(*tasks)
        return results

_ = await run_tasks(df.image_url)

Status code : 403 for https://images.gr-assets.com/books/1491494589m/18484598.jpg
Status code : 403 for https://images.gr-assets.com/books/1493667294m/35040602.jpg
Status code : 403 for https://images.gr-assets.com/books/1501972305m/35919935.jpg
